In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Objectif : à partir des sources disponibles constituer un fichier des consommations d'énergie , éléctricité 
#et gaz naturel à la maille EPCI.
#La définition des EPCI sera celle en vigueur au début 2023 ( mise à jour INSEE)
#Elle sera appliquée sur tout l'historique des données de manière à raisonner à périmètre stable

In [3]:
#Sources de données :Consommation annuelle d’électricité et gaz par IRIS et par secteur d’activité

#https://www.data.gouv.fr/fr/datasets/consommation-annuelle-delectricite-et-gaz-par-iris-et-par-secteur-dactivite/#/resources


#via Agence ORE
"""
L’Agence ORE regroupe tous les acteurs de la distribution d’électricité et de gaz en France (plus de 125 entités)
, qui coopèrent avec un objectif commun : répondre aux nouveaux enjeux numériques de la transition énergétique.
Association de service public, ses principales missions sont de :
   -Faciliter et fluidifier le fonctionnement des marchés de l’électricité et du gaz
   -Proposer une offre de données agrégées au service de la transition énergétique, à destination des acteurs des territoires
   et de l'innovation


Description du jeu de données:

Permet de visualiser l’évolution de 2011 à 2021 des consommations d'électricité et de gaz par secteur d'activité (résidentiel, 
tertiaire, industriel, agricole ou non affecté) et par IRIS.

Le jeu de données décrit aussi le nombre de points de livraison par maille géographique.

Les données sont ventilées sur le référentiel INSEE au 1er janvier 2021.

Ces données sont publiées dans le respect des règles relatives à la protection des Informations Commercialement Sensibles.

"""
conso_an_secteur=pd.read_csv(r"C:\DATA\énergie\conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-iris.csv",sep=";")


In [4]:
#Descriptif issu des méta-données

"""
Description du fichier

Opérateur: operateur[text] Nom de l’opérateur, qu’il soit distributeur ou transporteur

Année: annee[date] Année

Filière: filiere[text] Énergie concernée (Electricité ou Gaz)

Consommation Agriculture (MWh): consoa[double] Consommation dans l’agriculture (MWh). Nombre positif ou nul.

Nombre de points Agriculture: pdla[int] Nombre de points de livraison dans l’agriculture.
Nombre entier.

Nombre de mailles secretisées (agriculture): nombre_maille_secretisees_a[int] Cette colonne apporte une information sur l’écart
que la sécretisation a générée sur la somme de consommation du secteur agricole.
Ex. La consommation sur la maille commune est calculée comme la somme des consommations des IRIS composant la commune.
Si 2 mailles sont secretisées, l’agrégat de la commune contient la valeur “2” dans cette colonne.
Il est donc possible de déduire que l’erreur maximale sur la somme des consommations à cette maille est soit de 400 MWh
(200 MWh 2) soit 20 PDL (102).

Indique qualité Agriculture: indquala[double] Pourcentage de la consommation annuelle dans l’agriculture qui est mesurée du
1er janvier au 31 décembre. Il est compris entre 0 et 1.

Consommation Industrie (MWh): consoi[double] Consommation dans l’industrie (MWh). Nombre positif ou nul.

Nombre de points Industrie: pdli[int] Nombre de points de livraison dans l’industrie.
Nombre entier.

Nombre de mailles secretisées (industrie): nombre_maille_secretisees_i[int] Cette colonne apporte une information sur l’écart que la sécretisation a générée sur la somme de consommation du secteur industriel.
Ex. La consommation sur la maille commune est calculée comme la somme des consommations des IRIS composant la commune.
Si 2 mailles sont secretisées, l’agrégat de la commune contient la valeur “2” dans cette colonne.
Il est donc possible de déduire que l’erreur maximale sur la somme des consommations à cette maille est soit de 400 MWh (200 MWh 2) soit 20 PDL (102).

Indice qualité Industrie: indquali[double] Pourcentage de la consommation annuelle dans l’industrie qui est mesurée du 1er janvier au 31 décembre. Il est compris entre 0 et 1.

Consommation Tertiaire (MWh): consot[double] Consommation dans le tertiaire (MWh). Nombre positif ou nul.

Nombre de points Tertiaire: pdlt[int] Nombre de points de livraison dans le tertiaire.
Nombre entier.

Nombre de mailles secretisées (tertiaire): nombre_maille_secretisees_t[int] "Cette colonne apporte une information sur l’écart que la sécretisation a générée sur la somme de consommation du secteur tertiaire.
Ex. La consommation sur la maille commune est calculée comme la somme des consommations des IRIS composant la commune.
Si 2 mailles sont secretisées, l’agrégat de la commune contient la valeur “2” dans cette colonne.
Il est donc possible de déduire que l’erreur maximale sur la somme des consommations à cette maille est soit de 400 MWh (200 MWh 2) soit 20 PDL (102).

Indice qualité Tertiaire: indqualt[double] Pourcentage de la consommation annuelle dans le tertiaire qui est mesurée du 1er janvier au 31 décembre. Il est compris entre 0 et 1.

Consommation Résidentiel (MWh): consor[double] Consommation dans le résidentiel (MWh). Nombre positif ou nul.

Nombre de points Résidentiel: pdlr[int] Nombre de points de livraison dans le résidentiel. Nombre entier, positif ou nul.

Nombre de mailles secretisées (résidentiel): nombre_maille_secretisees_r[int] Cette colonne apporte une information sur l’écart que la sécretisation a générée sur la somme de consommation du secteur résidentiel.
Ex. La consommation sur la maille commune est calculée comme la somme des consommations des IRIS composant la commune.
Si 2 mailles sont secretisées, l’agrégat de la commune contient la valeur “2” dans cette colonne.
Il est donc possible de déduire que l’erreur maximale sur la somme des consommations à cette maille est soit de 400 MWh (200 MWh 2) soit 20 PDL (102).

Indice qualité Résidentiel: indqualr[double] Pourcentage de la consommation annuelle dans le résidentiel qui est mesurée du 1er janvier au 31 décembre. Il est compris entre 0 et 1.

Thermosensibilité (MWh/degré-jour): thermor[double] En MWh/degré-jour à la maille IRIS : variation de la consommation journalière en MWh due à une variation de 1°C de la température, pour les températures inférieures à la température seuil faisant référence pour l’IRIS considéré (voir note méthodologique).

Part Thermosensible: partr[double] En %, à la maille IRIS : pourcentage de la consommation totale que l’on considère sensible à la température.

Consommation Secteur Inconnu (MWh): consona[double] Consommation non affectée dans l’un des quatre autres secteurs (MWh). Nombre entier, positif ou nul.

Nombre de points Secteur Inconnu: pdlna[int] Nombre de points de livraison non affectés dans l’un des quatre autres secteurs. Nombre entier, positif ou nul.

Nombre de mailles secretisées (secteur inconnu): nombre_maille_secretisees_na[int] Cette colonne apporte une information sur l’écart que la sécretisation a générée sur la somme de consommation du secteur inconnu.
Ex. La consommation sur la maille commune est calculée comme la somme des consommations des IRIS composant la commune.
Si 2 mailles sont secretisées, l’agrégat de la commune contient la valeur “2” dans cette colonne.
Il est donc possible de déduire que l’erreur maximale sur la somme des consommations à cette maille est soit de 400 MWh (200 MWh 2) soit 20 PDL (102).

Indice qualité Non Affecté: indqualna[double] Pourcentage de la consommation annuelle dans le secteur inconnu qui est mesurée du 1er janvier au 31 décembre. Il est compris entre 0 et 1.

Code IRIS: code_iris[text] Code de l’IRIS

Libellé IRIS: libelle_iris[text] Libellé de l’IRIS

Code Commune: code_commune[text] Code INSEE de la commune

Libellé Commune: libelle_commune[text] Libellé de la commune

Code EPCI: code_epci[text] Code EPCI de la commune

Libellé EPCI: libelle_epci[text] Libellé EPCI de la commune

Code Département: code_departement[text] Code du département de la commune

Libellé Département: libelle_departement[text] Libellé du département de la commune

Code Région: code_region[int] Code Région de la commune

Libellé Région: libelle_region[text] Libellé Région de la commune

id_filiere: id_filiere[int] Identifiant de la filière d’énergie

Consommation Totale (MWh): consototale[double] Consommation totale (MWh)


SECRETISATION DES DONNEES ET UTILISATION DES LIGNES EPCI/REGION:

Les textes réglementaires prévoient que les données dans le secteur du résidentiel ne peuvent être 
diffusées si le nombre de points de livraison est inférieur à 11 et la consommation résidentielle inférieure 
ou égale à 200 MWh. 

Pour ces cas, la secrétisation est réalisée selon la procédure suivante : 
- la consommation du secteur résidentiel (CONSOR) prend la valeur -99 ;
- le nombre de points de livraison du résidentiel (PDLR) prend la valeur -99 ;
- le total des valeurs secrétisées est regroupé par intercommunalité et par région afin de permettre
de calculer des totaux pour ces deux mailles géographiques. Des lignes dont le type est N InterCom O 
(pour les EPCI) et N Region O (région) ont été ajoutées avec les totaux des IRIS ou communes secrétisés 
pour pouvoir faire des totaux à ces niveaux géographiques. 

Lorsqu’une ligne EPCI est présente, cela signifie qu’il y a des communes ou des IRIS qui la composent 
qui ont des données masquées. La ligne EPCI correspond à la somme de ces données masquées. 
Cependant, une ligne EPCI n’est pas diffusée si la somme de ces données masquées correspond à un 
nombre de points de livraison inférieur à 11 et à une consommation résidentielle inférieure ou égale à
200 MWh. 

Lorsqu’une ligne région est présente, cela signifie qu’il y a des communes ou des IRIS qui la 
composent qui ont des données masquées et que par ailleurs tout ou partie de ces données masquées 
n’ont pas pu être diffusées à l’échelon EPCI. La ligne région correspond alors à la somme de ces données 
masquées qui n’ont pas pu être diffusées à la maille EPCI. 

Pour calculer le total pour un EPCI, il faut sommer les agrégats non masqués des IRIS/communes qui 
composent cet EPCI et la ligne de cet EPCI (si elle n’est pas masquée elle-même). 
Pour calculer le total pour une région, il faut sommer les agrégats non masqués des IRIS/communes 
qui composent cette région et la ligne éventuelle de cette région. 
Pour calculer le total France entière, il faut sommer les agrégats non masqués des IRIS/communes, 
ainsi que les lignes éventuelles des EPCI et des régions. 


Définition Insee de l' EPCI :Établissement public de coopération intercommunale

"""

"""
Infos sur les secteurs :

Le code NAF à deux niveaux (divisions) se réfère à la nomenclature d'activités française révision 2
(NAF rév. 2, 2008) présentée sur le site de l’Insee : https://www.insee.fr/fr/information/2120875.
La correspondance entre la NAF à 2 niveaux (à 2 chiffres) et les grands secteurs d’activité est la
suivante.
Agriculture : 01, 02, 03
Industrie : 05, 06, 07, 08, 09, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
31, 32, 35, 41, 42, 43
Tertiaire : 33, 36, 37, 38, 39, 45, 46, 47, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68,
69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99


"""
conso_an_secteur=conso_an_secteur.rename(
    {
    "consoa":"conso_agri",
    "consot":"conso_ter",
    "consoi":"conso_indus",
    "consor":"conso_resid",
    "consona":"conso_nonaff",
    "consototale":"conso_totale"
    },
    axis=1)

#Mise au format des codes communes et codes département
conso_an_secteur['code_commune']=conso_an_secteur['code_commune'].apply(lambda x: "0"+str(x) if len(str(x))<5 else str(x))
conso_an_secteur['code_departement']=conso_an_secteur['code_departement'].apply(lambda x: str(x))


In [24]:
conso_an_secteur.columns

Index(['operateur', 'annee', 'filiere', 'conso_agri', 'pdla',
       'nombre_maille_secretisees_a', 'indquala', 'conso_indus', 'pdli',
       'nombre_maille_secretisees_i', 'indquali', 'conso_ter', 'pdlt',
       'nombre_maille_secretisees_t', 'indqualt', 'conso_resid', 'pdlr',
       'nombre_maille_secretisees_r', 'indqualr', 'thermor', 'partr',
       'conso_nonaff', 'pdlna', 'nombre_maille_secretisees_na', 'indqualna',
       'code_iris', 'libelle_iris', 'code_commune', 'libelle_commune',
       'code_epci', 'libelle_epci', 'code_departement', 'libelle_departement',
       'code_region', 'libelle_region', 'id_filiere', 'conso_totale'],
      dtype='object')

In [25]:
#Autres sources pour les informations communes et epci

#Liste officielle des communes : Code officiel Géographique
liste_commune=pd.read_csv(r"C:\DATA\contours_cartes\COG_insee_commune.csv")

#Liste officielle des ecpi avec la nature des epci ( 1ère feuille du fichier insee))
liste_epci=pd.read_excel(r"C:\DATA\contours_cartes\reference_Intercommunalite_Metropole_au_01-01-2023.ods",
                         sheet_name=0,header=5)



#Liste officielle des epci par commune
liste_commune_epci=pd.read_csv("C:\DATA\contours_cartes\liste_communes_epci.csv")
liste_commune_epci=liste_commune_epci[["code_commune","libelle_commune","code_epci"]]

#Informations surface et population des communes
donnees_communes=gpd.read_file(r"C:\DATA\contours_cartes\donnees_communes.geojson")
donnees_communes=donnees_communes[["INSEE_COM","superficie","P19_POP"]]

#Mise au format des différents codes

liste_epci["EPCI"]=liste_epci["EPCI"].astype("str")
liste_commune_epci["code_epci"]=liste_commune_epci["code_epci"].astype("str")
liste_commune_epci['code_commune']=liste_commune_epci['code_commune'].apply(lambda x: "0"+str(x) if len(str(x))<5 else str(x))

In [26]:
#Transformations et Vérifications du dataframe conso_an_secteur

In [27]:
#Côté "géographique":On ne conserve que les codes communes et IRIS avant de faire un premier niveau de regroupement par code commune
#Après filtrage des département 971 à 976
departements=list(["971","972","973","974","975","976"])
conso_an_secteur=conso_an_secteur[~conso_an_secteur["code_departement"].isin (departements)]
conso_an_secteur=conso_an_secteur.drop(["pdla",'nombre_maille_secretisees_a','indquala',
                                        'pdli','nombre_maille_secretisees_i', 'indquali',
                                        'pdlt','nombre_maille_secretisees_t', 'indqualt',
                                        'pdlr','nombre_maille_secretisees_r', 'indqualr', 'thermor', 'partr',
                                        'pdlna', 'nombre_maille_secretisees_na', 'indqualna',
                                       "libelle_iris","libelle_commune","code_epci","libelle_epci",
                                       "code_departement","libelle_departement","code_region","libelle_region" ],axis=1)

conso_an_secteur.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826594 entries, 0 to 830339
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   operateur     826594 non-null  object 
 1   annee         826594 non-null  int64  
 2   filiere       826594 non-null  object 
 3   conso_agri    826594 non-null  float64
 4   conso_indus   826594 non-null  float64
 5   conso_ter     826594 non-null  float64
 6   conso_resid   826594 non-null  float64
 7   conso_nonaff  826594 non-null  float64
 8   code_iris     826594 non-null  object 
 9   code_commune  826594 non-null  object 
 10  id_filiere    826594 non-null  int64  
 11  conso_totale  826594 non-null  float64
dtypes: float64(6), int64(2), object(4)
memory usage: 82.0+ MB


In [28]:
#Comparaison du total de conso totale avec la somme des totaux des conso par secteur
df_verif=conso_an_secteur[["conso_agri","conso_ter","conso_indus","conso_resid","conso_nonaff","conso_totale"]]

df_verif["verif_total"]=df_verif["conso_agri"]+df_verif["conso_ter"]+df_verif["conso_indus"]+df_verif["conso_resid"]+df_verif["conso_nonaff"]-df_verif["conso_totale"]

print("Total des écarts sur la vérification du total :",df_verif["verif_total"].sum())

#RAS

Total des écarts sur la vérification du total : -2.442252267087497e-08


In [29]:
#On calcul un total sur 11 ans des conso totales , pour voir ultérieurement l'effet des manipulations du dataframe
total_conso_2011_2021=conso_an_secteur["conso_totale"].sum()

print("La valeur du total de consommation en MWh est :",total_conso_2011_2021)

#RAS

La valeur du total de consommation en MWh est : 9754648754.5864


In [30]:
#regroupement par commune et filière
conso_an_secteur=conso_an_secteur.groupby(['annee', "code_commune",'filiere','id_filiere'],as_index=False).agg({"conso_agri":"sum","conso_ter":"sum",
                                                                        "conso_indus":"sum","conso_resid":"sum","conso_nonaff":"sum","conso_totale":"sum"})


In [31]:
conso_an_secteur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483200 entries, 0 to 483199
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   annee         483200 non-null  int64  
 1   code_commune  483200 non-null  object 
 2   filiere       483200 non-null  object 
 3   id_filiere    483200 non-null  int64  
 4   conso_agri    483200 non-null  float64
 5   conso_ter     483200 non-null  float64
 6   conso_indus   483200 non-null  float64
 7   conso_resid   483200 non-null  float64
 8   conso_nonaff  483200 non-null  float64
 9   conso_totale  483200 non-null  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 36.9+ MB


In [32]:
#Valeurs manquantes à ce stade
conso_an_secteur.isna().sum()

#Aucune valeur manquante

annee           0
code_commune    0
filiere         0
id_filiere      0
conso_agri      0
conso_ter       0
conso_indus     0
conso_resid     0
conso_nonaff    0
conso_totale    0
dtype: int64

In [33]:
#2ème  regroupement : 1 seule ligne  par commune et filière
#Les données filières ( électricité et gaz) se retrouvent en ligne
df=conso_an_secteur

#Ajout d'une variable "année-commune" pour la reconstitution ultérieure
df["annee"]=df["annee"].astype("str")
df["annee-commune"]=df["annee"]+"-"+df["code_commune"]
df=df.drop(["annee","code_commune"],axis=1)

#Dataframe electricité
df_elec=df[df["id_filiere"]==100]
df_elec=df_elec.drop(["filiere","id_filiere"],axis=1) 
for var in df_elec.columns[0:6]:
    df_elec=df_elec.rename({var:var+"_elec"},axis=1)

#Dataframe gaz
df_gaz=df[df["id_filiere"]==200]
df_gaz=df_gaz.drop(["filiere","id_filiere"],axis=1) 
for var in df_gaz.columns[0:6]:
    df_gaz=df_gaz.rename({var:var+"_gaz"},axis=1)
    
#Réinitialisation de  df avec les valeurs uniques de annee-commune pour la reconstitution)
df=df[["annee-commune"]].drop_duplicates()

df=df.merge(right=df_elec,on="annee-commune",how="left")
df=df.merge(right=df_gaz,on="annee-commune",how="left")

#les valeurs manquantes issues des fusions sont remplacées par 0
df=df.fillna(0)

#Ajout des totaux electricité + gaz
df["conso_agri"]=df["conso_agri_elec"]+df["conso_agri_gaz"]
df["conso_ter"]=df["conso_ter_elec"]+df["conso_ter_gaz"]
df["conso_indus"]=df["conso_indus_elec"]+df["conso_indus_gaz"]
df["conso_resid"]=df["conso_resid_elec"]+df["conso_resid_gaz"]
df["conso_nonaff"]=df["conso_nonaff_elec"]+df["conso_nonaff_gaz"]
df["conso_totale"]=df["conso_totale_elec"]+df["conso_totale_gaz"]

#Dernière opération : redécommposition de annee-commune
df["année"]=df["annee-commune"].apply(lambda x:x.split("-")[0])
df["code_commune"]=df["annee-commune"].apply(lambda x:x.split("-")[1])


#Vérification sur la conso totale 
print ("Consommation totale (valeur attendue  9754648754.5864):",df["conso_totale"].sum())

#Conclusion RAS , pas de perte d'infos

Consommation totale (valeur attendue  9754648754.5864): 9754648754.586397


In [34]:
#3ième opération : on ramène les infos communes et epci
#infos communes
df=df.merge(right=liste_commune_epci,on="code_commune",how="left")

#il manque des infos sur 18 communes , on enlève les lignes
df=df.dropna()
#Nouveau_total
total=df["conso_totale"].sum()

print("Le total avant suppression des lignes est en MWh :",total_conso_2011_2021)
print("Le total après suppression des lignes est en MWh :",total)
print("L'écart avec le total initial est :",total_conso_2011_2021-total," soit ",(total_conso_2011_2021-total)/total_conso_2011_2021*100 , "%")

#RAS

Le total avant suppression des lignes est en MWh : 9754648754.5864
Le total après suppression des lignes est en MWh : 9754635693.626398
L'écart avec le total initial est : 13060.960000991821  soit  0.0001338947237321167 %


In [35]:
#Infos epci
df=df.merge(right=liste_epci,right_on="EPCI",left_on="code_epci",how="left")
df=df.drop("EPCI",axis=1)

#RAS

In [36]:
#Infos population et superficie
df=df.merge(right=donnees_communes,right_on="INSEE_COM",left_on="code_commune",how="left")
df=df.rename({"LIBEPCI":"nom_epci","NATURE_EPCI":"nature_epci","NB_COM":"nb_com","P19_POP":"population"},axis=1)
df=df.drop(["annee-commune","INSEE_COM"],axis=1)

#il manque des informations sur 10 communes
#'02695': Saint-Thibaut dans l'Aisne --> rattachée maintenant  Bazoches-et-Saint-Thibaut, depuis le 1er janvier 2022.
#16010': Ambleville au  01/01/2022 : Ambleville devient commune déléguée au sein de Lignières-Ambleville (16186) (commune nouvelle).
#'19092': Jardin au 01/01/2022 : Le Jardin est rattachée à Montaignac-Saint-Hippolyte (19143) qui devient Montaignac-sur-Doustre (commune nouvelle).
#'24089':01/01/2022 : Cazoulès devient commune déléguée au sein de Pechs-de-l'Espérance (24325) (commune nouvelle).
#'24314':01/01/2022 : Orliaguet devient commune déléguée au sein de Pechs-de-l'Espérance (24325) (commune nouvelle).
#'25134':01/01/2022 : Châtillon-sur-Lison est rattachée à Cussey-sur-Lison (25185) (commune nouvelle).
#'25628':01/01/2022 : Villers-sous-Montrond est rattachée à Mérey-sous-Montrond (25375) qui devient Les Monts-Ronds (commune nouvelle)
#'26219':01/01/2022 : Mureils devient commune déléguée au sein de Saint-Jean-de-Galaure (26216) (commune nouvelle)
#'56049':01/01/2022 : Croixanvec devient commune déléguée au sein de Saint-Gérand-Croixanvec (56213) (commune nouvelle)
#'85307':01/01/2022 : La Faute-sur-Mer devient commune déléguée au sein de L'Aiguillon-la-Presqu'île (85001) (commune nouvelle).

# on met les 2 paramètres à 0 pour ces communes , parce qu'elles sont déjà comprises dans les communes nouvelles.
#Ce sera neutre au moment de l'aggrégation au niveau des EPCI


df=df.fillna(0)


In [37]:
df.isna().sum()

conso_agri_elec      0
conso_ter_elec       0
conso_indus_elec     0
conso_resid_elec     0
conso_nonaff_elec    0
conso_totale_elec    0
conso_agri_gaz       0
conso_ter_gaz        0
conso_indus_gaz      0
conso_resid_gaz      0
conso_nonaff_gaz     0
conso_totale_gaz     0
conso_agri           0
conso_ter            0
conso_indus          0
conso_resid          0
conso_nonaff         0
conso_totale         0
année                0
code_commune         0
libelle_commune      0
code_epci            0
nom_epci             0
nature_epci          0
nb_com               0
superficie           0
population           0
dtype: int64

In [38]:
#Export du fichier

df.to_csv(r"C:\DATA\énergie\conso_an_secteur.csv",index=False)